<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5d0debdee8b9b00f7f05ba17d3a0beceaf4a7e11664b20bdb6cf7e1a07a40395
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 10:25:41
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -87.06 K (-0.61%)
Current PnL: -24.27 L (-15.91%)
CY Booked + Current PnL: -10.38 L (-6.8%)
-------------------
Total profit:  1.39 L
Total loss:  -25.67 L
-------------------
Total Booked + Current PnL: 16.60 L (13.21%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.68 L (63.37%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.30,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.01,-10.05,11.20,0.02,17585.0,-17550.0,157009.0,139.40,62.0,M-SC,14.18,234.0,-1.00,1.11,36.57,OX40N,NTT,PAINTS
51,MASFIN,398.61,-0.08,-2.63,25.35,22.05,24184.0,-2580.0,95400.0,-15.99,58.0,H-SC,6.71,164.0,-0.11,0.67,38.67,XR,ATH,FINANCE
43,ITC,452.00,-0.07,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.34,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.34,4.57,14.94,20.19,27234.0,7969.0,182289.0,-18.14,43.0,X-MC,6.69,68.0,0.29,1.29,20.38,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,11.55,-43.43,198.81,69.05,149664.0,-57786.0,75280.0,-54.32,64.0,H-SC,23.54,140.0,-0.39,0.53,16.10,SR,ATH,IT
13,BSOFT,831.70,2.70,-24.60,99.98,50.79,106033.0,-34599.0,106054.0,-3.36,69.0,H-SC,5.91,151.0,-0.33,0.75,23.48,XR,ATH,IT
85,WIPRO,420.00,1.90,4.56,64.76,72.27,114735.0,7721.0,177169.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,11.01,XR,NTT,IT
74,TCS,4389.97,1.45,-13.56,37.99,19.28,113606.0,-46914.0,299042.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.11,10.55,X40,ATH,IT
9,BAJAJHFL,181.50,1.30,-19.61,84.68,48.45,136486.0,-39327.0,161179.0,-25.46,21.0,X-MC,8.80,64.0,-0.29,1.14,1.30,X40N,ATH,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-3.31,-38.19,82.66,12.90,111636.0,-83442.0,135054.0,99.52,49.0,H-SC,12.09,161.0,-0.75,0.95,12.30,OX40N,NTT,FINANCE
52,MEDANTA,1486.00,-2.35,-0.15,24.27,24.08,30763.0,-195.0,126755.0,-8.22,38.0,X-SC,5.86,89.0,-0.01,0.90,19.43,XY24,NTT,HEALTHCARE
24,EASEMYTRIP,26.40,-2.34,-53.97,216.55,45.70,176233.0,-95433.0,81382.0,-15.39,65.0,M-SC,23.36,205.0,-0.54,0.58,17.30,XY24,NTT,TRAVEL
84,WHIRLPOOL,2270.00,-2.29,-20.92,130.04,81.92,118058.0,-24012.0,90786.0,-53.62,22.0,M-SC,4.95,236.0,-0.20,0.64,7.48,XR,NTT,DURABLES
48,KPIGREEN,730.66,-2.05,-15.31,73.51,46.95,78007.0,-19180.0,106117.0,-21.84,27.0,H-SC,2.52,154.0,-0.25,0.75,32.94,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.04,-0.08,106.89,106.72,131727.0,-100.0,123236.0,-50.44,39.0,H-SC,1.71,139.0,-0.00,0.87,23.28,AR,ATH,MISC
37,IEX,219.00,-1.67,-0.88,49.94,48.63,98983.0,-1751.0,198203.0,-34.11,59.0,H-SC,14.82,136.0,-0.02,1.40,10.58,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.67,-0.59,36.43,35.63,75897.0,-1237.0,208337.0,-14.53,46.0,H-MC,3.66,119.0,-0.02,1.47,15.98,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.25,-5.89,120.94,107.91,168035.0,-8703.0,138941.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.01,-10.05,11.20,0.02,17585.0,-17550.0,157009.0,139.40,62.0,M-SC,14.18,234.0,-1.00,1.11,36.57,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.30,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.10,-22.94,47.26,13.47,98207.0,-61866.0,207801.0,-68.31,32.0,H-SC,2.14,158.0,-0.63,1.47,5.66,XY24,NTT,PAINTS
67,SIS,528.00,1.20,-24.38,60.36,21.26,50678.0,-27073.0,83959.0,1980.31,47.0,H-SC,3.90,166.0,-0.53,0.59,13.44,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.9,4.56,64.76,72.27,114735.0,7721.0,177169.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,11.01,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.90,4.56,64.76,72.27,114735.0,7721.0,177169.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,11.01,XR,NTT,IT
38,INDIAMART,4810.62,-0.04,-0.08,106.89,106.72,131727.0,-100.0,123236.0,-50.44,39.0,H-SC,1.71,139.0,-0.00,0.87,23.28,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.67,-0.59,36.43,35.63,75897.0,-1237.0,208337.0,-14.53,46.0,H-MC,3.66,119.0,-0.02,1.47,15.98,AR,ATH,PHARMA
37,IEX,219.00,-1.67,-0.88,49.94,48.63,98983.0,-1751.0,198203.0,-34.11,59.0,H-SC,14.82,136.0,-0.02,1.40,10.58,XR,NTT,MISC
25,FINCABLES,1641.55,-0.25,-5.89,120.94,107.91,168035.0,-8703.0,138941.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,1.30,-19.61,84.68,48.45,136486.0,-39327.0,161179.0,-25.46,21.0,X-MC,8.80,64.0,-0.29,1.14,1.30,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.73,-37.32,115.70,35.20,93313.0,-48019.0,80651.0,5.70,22.0,X-SC,14.22,92.0,-0.51,0.57,0.73,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.56,-21.51,95.56,53.50,226324.0,-64911.0,236840.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.20,XY24,NTT,FMCG
55,PGHH,17907.65,-0.61,-5.71,41.87,33.76,79279.0,-11475.0,189345.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
22,DIXON,18931.72,-1.88,-8.67,33.13,21.58,42403.0,-12150.0,127989.0,-53.98,30.0,X-MC,5.84,56.0,-0.29,0.90,13.31,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.65,-22.68,112.31,64.15,206628.0,-53971.0,183980.0,-54.57,46.0,X-SC,1.45,82.0,-0.26,1.30,3.20,XY24,BTT,CEMENT
43,ITC,452.0,-0.07,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.34,X40,NTT,FMCG
15,CAMS,4762.0,-0.14,0.77,23.83,24.79,60481.0,1947.0,253803.0,-3.91,46.0,X-SC,1.89,86.0,0.03,1.79,24.09,X40N,NTT,MISC
8,AWL,485.0,-0.56,-21.51,95.56,53.50,226324.0,-64911.0,236840.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.20,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.92,1.27,29.33,30.97,52495.0,2241.0,178979.0,-22.52,47.0,X-MC,2.17,75.0,0.04,1.26,13.99,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-1.58,-19.35,75.46,41.51,160277.0,-50965.0,212400.0,-3.88,34.0,X-MC,7.28,61.0,-0.32,1.50,0.00,XY25,ATH,FMCG
55,PGHH,17907.65,-0.61,-5.71,41.87,33.76,79279.0,-11475.0,189345.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
56,QUESS,424.00,-0.31,-29.68,103.13,42.84,47143.0,-19294.0,45712.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.08,-0.16,-8.28,37.45,26.07,56242.0,-13560.0,150180.0,-32.40,30.0,X-MC,10.46,55.0,-0.24,1.06,0.37,X40,ATH,APPARELS
31,HAVELLS,2069.16,-0.77,-9.63,46.59,32.48,138100.0,-31576.0,296415.0,-14.20,36.0,X-MC,4.96,67.0,-0.23,2.09,0.70,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.31,-29.68,103.13,42.84,47143.0,-19294.0,45712.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.27,-47.53,190.26,52.31,144917.0,-68992.0,76168.0,-44.15,35.0,X-SC,5.02,91.0,-0.48,0.54,1.96,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.73,-37.32,115.70,35.20,93313.0,-48019.0,80651.0,5.70,22.0,X-SC,14.22,92.0,-0.51,0.57,0.73,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.88,-8.67,33.13,21.58,42403.0,-12150.0,127989.0,-53.98,30.0,X-MC,5.84,56.0,-0.29,0.90,13.31,X40N,ATH,IT
52,MEDANTA,1486.00,-2.35,-0.15,24.27,24.08,30763.0,-195.0,126755.0,-8.22,38.0,X-SC,5.86,89.0,-0.01,0.90,19.43,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.45,-13.56,37.99,19.28,113606.0,-46914.0,299042.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.11,10.55,X40,ATH,IT
41,INFY,2275.00,-0.19,6.27,45.74,54.87,152082.0,19605.0,332493.0,-17.09,60.0,X-LC,2.48,2.0,0.13,2.35,15.07,X40,BTT,IT
76,TMPV,600.00,-1.11,-15.23,67.71,42.18,158662.0,-42084.0,234326.0,-24.30,32.0,X-LC,3.12,3.0,-0.27,1.66,1.50,XY24,NTT,AUTO
82,VBL,671.64,-1.15,-3.88,41.15,35.67,124928.0,-12250.0,303592.0,-15.30,57.0,X-LC,5.40,4.0,-0.10,2.15,9.44,X40N,ATH,FMCG
43,ITC,452.00,-0.07,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.34,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.78,-16.40,117.32,81.67,92005.0,-15388.0,78422.0,7050.00,40.0,L-SC,18.67,271.0,-0.17,0.55,51.87,XR,NTT,CERAMICS
51,MASFIN,398.61,-0.08,-2.63,25.35,22.05,24184.0,-2580.0,95400.0,-15.99,58.0,H-SC,6.71,164.0,-0.11,0.67,38.67,XR,ATH,FINANCE
13,BSOFT,831.70,2.70,-24.60,99.98,50.79,106033.0,-34599.0,106054.0,-3.36,69.0,H-SC,5.91,151.0,-0.33,0.75,23.48,XR,ATH,IT
70,SURYODAY,216.00,-0.35,-19.24,52.35,23.03,75706.0,-34455.0,144616.0,57.89,49.0,H-SC,10.00,167.0,-0.46,1.02,43.49,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.01,-10.05,11.20,0.02,17585.0,-17550.0,157009.0,139.40,62.0,M-SC,14.18,234.0,-1.00,1.11,36.57,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.01,-10.05,11.20,0.02,17585.0,-17550.0,157009.0,139.40,62.0,M-SC,14.18,234.0,-1.00,1.11,36.57,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.22,3.04,17.38,20.95,45088.0,7656.0,259424.0,2.04,78.0,X-LC,17.33,31.0,0.17,1.83,39.76,X40,ATH,PAINTS
13,BSOFT,831.70,2.70,-24.60,99.98,50.79,106033.0,-34599.0,106054.0,-3.36,69.0,H-SC,5.91,151.0,-0.33,0.75,23.48,XR,ATH,IT
50,LTIM,7201.88,0.33,13.06,16.45,31.66,45781.0,32151.0,278302.0,8.61,70.0,H-LC,14.95,43.0,0.70,1.97,54.42,X200,ATH,IT
66,SIEMENS,4671.50,-0.87,-10.48,40.21,25.51,66986.0,-19505.0,166590.0,2.20,64.0,H-LC,2.71,49.0,-0.29,1.18,22.76,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.02
1,25,44.78
2,50,75.60


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    28.90
LC    25.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.61
X40      21.29
X40N     11.92
XR        9.67
AR        9.10
XY25      9.06
OX40N     7.59
X200      1.97
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.95
X-MC    22.44
X-LC    20.30
M-SC    11.91
X-SC     8.17
H-MC     4.75
H-LC     3.15
L-SC     1.38
M-MC     1.37
M-LC     1.25
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.69,-6.51,40.96
IT,14.15,-12.89,71.17
FINANCE,9.34,-17.19,66.71
MISC,7.41,-23.73,74.42
PAINTS,6.00,-8.77,24.93
ELECTRICAL,5.99,-10.56,51.06
INSURANCE,4.37,-2.12,38.11
PHARMA,3.90,-1.16,33.53
AUTO,3.37,-20.81,72.61


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3132878.0,21
XR,1291535.0,13
AR,1284241.0,10
X40,1023904.0,14
X40N,761534.0,9
OX40N,717704.0,10
XY25,358669.0,6
SR,274215.0,2
MH,78007.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3588169.0,25
M-SC,1376188.0,15
X-MC,1306016.0,16
X-LC,889309.0,11
X-SC,763879.0,8
H-MC,405302.0,3
L-SC,265291.0,3
M-LC,114735.0,1
H-LC,112767.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247224.0      6
           AR         882822.0      5
M-SC       XY24       784222.0      6
H-SC       XR         775408.0      7
X-MC       X40        462656.0      7
           XY24       390036.0      3
X-LC       X40        380330.0      5
H-SC       OX40N      330493.0      4
M-SC       OX40N      300808.0      5
X-SC       X40N       298427.0      3
           XY24       284534.0      3
H-SC       SR         274215.0      2
X-MC       X40N       264475.0      4
X-LC       XY24       232763.0      2
H-MC       AR         211203.0      2
X-LC       X40N       198632.0      2
H-MC       XY24       194099.0      1
X-MC       XY25       188849.0      2
X-SC       X40        180918.0      2
L-SC       XR         178888.0      2
M-SC       XR         167928.0      2
           AR         123230.0      2
M-LC       XR         114735.0      1
L-SC       OX40N       86403.0      1
H-SC       MH          78007.0      1
X-LC       XY25        77584.0      2
H-LC       AR          66986.0      1
L-MC       XR          54576.0      1
M-MC       XY25        51159.0      1
H-LC       X200        45781.0      1
L-LC       XY25        41077.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
